In [84]:
import pandas as pd
import numpy as np
import dask.dataframe as ddf
import re, nltkm 
from numpy import array
from pymongo import MongoClient

# tensorflow imports
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Activation,
    Dropout,
    Dense,
    Flatten,
    GlobalMaxPooling1D,
    Embedding,
    LSTM
)
from tensorflow.keras.initializers import RandomUniform, glorot_uniform
from tensorflow.keras.optimizers import Adam
# from Tkeras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

## Collect all Data

In [2]:
# get data from database
client = MongoClient('localhost:27017')
db = client.hotels
collection = db.reviews.find()

# transform to pandas dataframe
all_reviews = pd.DataFrame(list(collection), columns=['review', 'positive'])

## Function to pre proces text

In [59]:
# this function is used for apply
def proces_text(row):
    
    the_text = row['review']

    row['review'] = proces_text_only(the_text)    
    
    return row

# this function is used for only text
def proces_text_only(text_only):
    # Remove all the special characters (pro_fea = processed feature)
    pro_fea = re.sub(r'\W', ' ', text_only)
    # remove all single characters
    pro_fea = re.sub(r'\s+[a-zA-Z]\s+', ' ', pro_fea)
    # Remove single characters from the start
    pro_fea = re.sub(r'\^[a-zA-Z]\s+', ' ', pro_fea) 
    # Substituting multiple spaces with single space
    pro_fea = re.sub(r'\s+', ' ', pro_fea, flags=re.I)
    # Removing prefixed 'b'
    pro_fea = re.sub(r'^b\s+', '', pro_fea)
    # Converting to Lowercase
    return pro_fea.lower()    

## Apply all the function to all rows (( maybe apply dask))

In [4]:
# proc_rev = all_reviews.apply(proces_text, axis=1)
ddf_reviews = ddf.from_pandas(all_reviews, npartitions=7)
ddf_rev_pr = ddf_reviews.apply(proces_text, axis=1, meta={'review': 'object', 'positive': 'int64'})

In [5]:
df_rev = ddf_rev_pr.compute()

## Set Axis info

In [13]:
REV_LIMIT = 100000
rev_negatives = df_rev[df_rev.positive == 0][:REV_LIMIT]
rev_positives = df_rev[df_rev.positive == 1][:REV_LIMIT]

rev_balanced = pd.concat([rev_negatives, rev_positives]).reset_index(drop=True)

X = np.array(list(rev_balanced.loc[:, 'review']))
y = np.array(list(rev_balanced.loc[:, 'positive']))

In [7]:
len(y)

775696

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=42)

# ADD EXPLANATION HERE

In [87]:
max_words=5000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# ADD EXPLANATION HERE

In [16]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# ADD EXPLANATION HERE

In [17]:
# 2. define model
e_init = RandomUniform(-0.01, 0.01, seed=1)
init = glorot_uniform(seed=1)
simple_adam = Adam()
embed_vec_len = 32  # values per word

In [18]:
model = Sequential()

model.add(
    Embedding(
        input_dim=max_words,
        output_dim=embed_vec_len,
        embeddings_initializer=e_init,
        mask_zero=True
    )
)

model.add(
    LSTM(
        units=100,
        kernel_initializer=init,
        dropout=0.2,
        recurrent_dropout=0.2
    )
)

model.add(
    Dense(
        units=1,
        kernel_initializer=init,
        activation='sigmoid'
    )
)

# ADD EXPLANATION HERE

In [19]:
model.compile(
    loss='binary_crossentropy',
    optimizer=simple_adam,
    metrics=['acc']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [20]:
bat_size = 32
max_epochs = 15

model.fit(
    X_train,
    y_train,
    epochs=max_epochs,
    batch_size=bat_size,
    shuffle=True,
    verbose=1
) 


Starting training 
Train on 160000 samples
Epoch 1/3
160000/160000 [==============================] - 349s 2ms/sample - loss: 0.1844 - acc: 0.9358
Epoch 2/3
160000/160000 [==============================] - 345s 2ms/sample - loss: 0.1468 - acc: 0.9482
Epoch 3/3
160000/160000 [==============================] - 344s 2ms/sample - loss: 0.1322 - acc: 0.9545
Training complete 



In [21]:
loss_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test data: loss = %0.6f  accuracy = %0.2f%% " % \
  (loss_acc[0], loss_acc[1]*100))

Test data: loss = 0.134788  accuracy = 95.31% 


## Save mode + weights to disk

In [83]:
# serialize model to JSON
# model_json = model.to_json()
# with open("rev_sent_model.json", "w") as json_file:
#     json_file.write(model_json)
# serialize weights to HDF5
# model.save_weights("rev_sent_model.h5")
# print("Saved model to disk")

Saved model to disk


## Testing on single review

In [80]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

example_review = df_rev.iloc[220]['review']
example_review = proces_text_only(example_review)
stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(example_review)

filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w)

the_rev = tokenizer.texts_to_sequences(filtered_sentence)

ent_list = []

for sublist in the_rev:
    for item in sublist:
        ent_list.append(item)

instance = pad_sequences([ent_list], padding='post', maxlen=maxlen)

print('This review is positive') if model.predict(instance)[0][0] > 0.5 else print('This review is negative')
print(model.predict(instance)[0][0])

This review is negative
0.0049086213


## Model statistics

In [88]:
# import matplotlib.pyplot as plt

# plt.plot(sentimentor.history['acc'])
# plt.plot(sentimentor.history['val_acc'])

# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train','test'], loc='upper left')
# plt.show()

# plt.plot(sentimentor.history['loss'])
# plt.plot(sentimentor.history['val_loss'])

# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train','test'], loc='upper left')
# plt.show()